In [ ]:
import json
import sys

sys.path.append('..')
from tools.TextProcessing import build_word_tree, process_keywords, batched_sent_tokenize, clean_text
from tools.BasicUtils import my_write, my_csv_read, MultiThreading, get_wikipedia_entity

<h2> Select keyword file and generate keyword list

In [ ]:
# Collect keywords from terms-cs-cfl-epoch200.txt
stable_kw = []
unstable_kw = []
r = my_csv_read('../data/raw_data/terms-cs-cfl-epoch200.txt', delimiter='\t')
candidate_kw_list = [item[0] for item in r if float(item[1]) > 0.1 and item[2] == '1']
stable_kw, unstable_kw = process_keywords(candidate_kw_list)

<h2> Bind word with wikipedia page

In [ ]:
# Collect keywords that can be linked to a wikipedia page (11 minutes)
mt = MultiThreading()
ret = mt.run(lambda x: (x, get_wikipedia_entity(x)), stable_kw, thread_num=10)
kw2ent_dict = {item[0]:item[1] for item in ret if item[1] is not None}
json.dump(kw2ent_dict, open('../data/corpus/kw_ent_map.json', 'w'))
final_keywords = list(kw2ent_dict.keys())

<h2> Store the keyword list and build word tree

In [ ]:
# Save keywords
my_write('../data/corpus/keyword_f.txt', final_keywords)
# Generate word tree (25 seconds)
build_word_tree('../data/corpus/keyword_f.txt', '../data/corpus/wordtree.json', '../data/corpus/entity.txt')

<h2> Process sentences

In [ ]:
# Extract sentence from small corpus (2 minute)
j = json.load(open('../data/raw_data/small_arxiv.json', 'r'))
sents = [clean_text(i['abstract']) for i in j]
sents = batched_sent_tokenize(sents)
my_write('../data/corpus/small_sent.txt', sents)

In [ ]:
# Generate Reformed sentences (3 minutes)
# To run the code in the backend, use the gen_reform.py in the "py" folder

<h2> Collect wikipedia pages for keywords

In [ ]:
# To run the code in the backend, use the collect_wiki_page.py in the "py" folder